In [1]:
%gui wx
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
#from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ
# To be modified: accessing utils

In [2]:
async def importFSLasync():
    #load fsl module
    import lmod
    import os
    await lmod.purge(force=True)
    await lmod.load('fsl/6.0.7.4')
    await lmod.list()

def loadFSL():
    """
    Function to load FSL 6.0.7.4 module
    Ensures proper environment variables are setup.
    This function should be called at the start of any
    notebook which relies in any capacity on FSL.
    
    If you wish to change the FSL version being used,
    you should edit within the load AND FSLDIR the version.
    Make sure it exists in the neurodocker image before changing it!
    """
    import asyncio
    import os
    # We need to do the import asynchronously, as modules rely on await within
    #asyncio.run(importFSLasync())
    os.environ["FSLDIR"]="/cvmfs/neurodesk.ardc.edu.au/containers/fsl_6.0.7.4_20231005/fsl_6.0.7.4_20231005.simg/opt/fsl-6.0.7.4/"
    os.environ["FSLOUTPUTTYPE"]="NIFTI_GZ"
    os.environ["SINGULARITY_BINDPATH"]="/data,/neurodesktop-storage,/tmp,/cvmfs"


def mkdir_no_exist(path):
    """
    Function to create a directory if it does not exist already.

    Parameters
    ----------
    path: string
        Path to create
    """
    import os
    import os.path as op
    if not op.isdir(path):
        os.makedirs(path)

class FSLeyesServer:
    """
    An FSL eyes class to manipulate frame and context across notebook.
    
    
    Attributes
    ----------
    overlayList: object
        List of overlays, the images displayed within FSLeyes. Each image is a separate overlay
    displayCtx: object
        Display context
    frame: object
        Frame used to display and interact with FSLeyes
    ortho: object
        View panel in orthographic mode
        
    Examples
    --------
    >>>> %gui wx # Do not forget to use this in a notebook!
    >>>> from utils import FSLeyesServer
    >>>> fsleyesDisplay = FSLeyesServer()
    >>>> fsleyesDisplay.show()
    
    It is not a server in the most proper sense, but merely a convenience wrapper.
    Before initializing this class, always make sure you call %gui wx in a cell of the notebook
    to enable GUI integration.
    """
    def __init__(self):
        import fsleyes
        from fsleyes.views.orthopanel import OrthoPanel
        overlayList, displayCtx, frame = fsleyes.embed()
        ortho = frame.addViewPanel(OrthoPanel)
        self.overlayList = overlayList
        self.displayCtx = displayCtx
        self.frame = frame
        self.ortho = ortho
    def show(self):
        """
        Show the current frame interactively.
        """
        self.frame.Show()

    def setOverlayCmap(self,overlayNbr,cmap):
        self.displayCtx.getOpts(self.overlayList[overlayNbr]).cmap = 'Render3'
    
    def resetOverlays(self):
        """
        Remove all overlays from the current frame
        """
        from fsleyes.views.orthopanel import OrthoPanel
    
        while len(self.overlayList) > 0:
            del self.overlayList[0]

        self.frame.removeViewPanel(self.frame.viewPanels[0])
        # Put back an ortho panel in our viz for future displays
        self.frame.addViewPanel(OrthoPanel)
    
    def load(self,image_path):
        """
        Add a Nifti image to the current frame as a new overlay

        Parameters
        ----------
        image_path:  string
            Path to the Nifti image to add to the frame.
            
        Example
        -------
        >>>> %gui wx # Do not forget to use this in a notebook!
        >>>> from utils import FSLeyesServer
        >>>> fsleyesDisplay = FSLeyesServer() 
        >>>> fsleyesDisplay.load(op.expandvars('$FSLDIR/data/standard/MNI152_T1_0.5mm'))
        >>>> fsleyesDisplay.show()
        """
        from fsl.data.image import Image
        import fsleyes.data.tractogram as trk
        if ".trk" in image_path:
            trk_overlay = trk.Tractogram(image_path)
            self.overlayList.append(trk_overlay)
        else:
            self.overlayList.append(Image(image_path))
    def close(self):
        """
        Closes the server and free up resources.
        """
        import fsleyes
    
        self.frame.Close()
        fsleyes.shutdown()

def fsleyes_thread():
    """
    Function to run the FSLeyesServer in a separate thread.
    This function keeps the server running indefinitely.
    """
    fsleyesDisplay = FSLeyesServer()
    fsleyesDisplay.show()
    
    # Keep the thread alive
    while True:
        time.sleep(1)


def get_json_from_file(fname):
    """
    Given a filename pointing to a json, returns the json's content.

    Parameters
    ----------
    fname: string
        The filename of the json file

    Returns
    -------
    The data of the json file
    """
    import json
    f = open(fname)
    data = json.load(f)
    f.close()
    return data


In [3]:

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################
os.environ["DIPY_HOME"] = "/home/jovyan/Data"


#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri

import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json
import subprocess

In [4]:
################
# Start FSLeyes (very neat tool to visualize MRI data of all sorts) within Python
################
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

Gtk-Message: 10:06:23.339: Failed to load module "canberra-gtk-module"

(ipykernel_launcher.py:717): Gtk-CRITICAL **: 10:06:23.903: gtk_window_resize: assertion 'height > 0' failed


## 1.1 Downloading Data

In [5]:
dataset_id = 'ds000171'
subject = 'control01' 

sample_path = "/home/jovyan/Data/dataset"
mkdir_no_exist(sample_path)
bids_root = op.join(sample_path, dataset_id)
deriv_root = op.join(bids_root, 'derivatives')
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')

mkdir_no_exist(bids_root)

subject_dir = 'sub-{}'.format(subject)

###################
# Openneuro download.
###################
subprocess.run(["openneuro-py", "download", "--dataset", dataset_id, # Openneuro has for each dataset a unique identifier
                "--target-dir", bids_root,  # The path where we want to save our data. You should save your data under /home/jovyan/Data/[your dataset ID] to be 100% fool-proof
                "--include", subject_dir # Effectively get all data
               ], check=True)

#subprocess.run(["openneuro-py", "download", "--dataset", dataset_id, # Openneuro has for each dataset a unique identifier
#                "--target-dir", bids_root,  # The path where we want to save our data. You should save your data under /home/jovyan/Data/[your dataset ID] to be 100% fool-proof
#                "--include", op.join(subject_dir, 'anat','*'),# We are asking to get all files within the subject_dir/anat folder by using the wildcard *
#               "--include", op.join(subject_dir, 'func','sub-{}_task-sitrep_run-01_bold.*'.format(subject)),
#               ], check=True)

###################
# Create folders relevant for preprocessing.
# In BIDs, ANYTHING we modify must go in the derivatives folder, to keep original files clean in case we make a mistake.
###################
mkdir_no_exist(op.join(bids_root, 'derivatives'))
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')
mkdir_no_exist(preproc_root)
mkdir_no_exist(op.join(preproc_root, 'sub-control01'))
mkdir_no_exist(op.join(preproc_root, 'sub-control01', 'anat'))
mkdir_no_exist(op.join(preproc_root, 'sub-control01', 'func'))
mkdir_no_exist(op.join(preproc_root, 'sub-control01', 'fmap')) # Useful or not ?


👋 Hello! This is openneuro-py 2024.2.0. Great to see you! 🤗

   👉 Please report problems 🤯 and bugs 🪲 at
      https://github.com/hoechenberger/openneuro-py/issues

🌍 Preparing to download ds000171 …


╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /opt/conda/lib/python3.11/site-packages/openneuro/_cli.py:64 in download_cli │
│                                                                              │
│    61 │   ] = 5,                                                             │
│    62 ) -> None:                                                             │
│    63 │   """Download datasets from OpenNeuro."""                            │
│ ❱  64 │   download(                                                          │
│    65 │   │   dataset=dataset,                                               │
│    66 │   │   tag=tag,                                                       │
│    67 │   │   target_dir=target_dir,                                         │
│                                                                              │
│ ╭──────────────────────────── locals ─────────────────────────────╮          │
│ │                  dataset

CalledProcessError: Command '['openneuro-py', 'download', '--dataset', 'ds000171', '--target-dir', '/home/jovyan/Data/dataset/ds000171', '--include', 'sub-control01']' returned non-zero exit status 1.

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(op.join(bids_root, 'sub-control01', 'anat', 'sub-control01_T1w.nii.gz'))

## 1.2 Standardizing & Concatenating Data

In [ ]:
# Define the path to your fMRI runs and load each run
fmri_paths = glob.glob("/path_to_your_fmri_runs/sub-*/func/sub-*_task-*run-*_bold.nii.gz")

# List to hold the standardized data arrays from each run
standardized_fmri_data = []

# Loop through each run file and load it
for run_path in fmri_paths:
    img = nib.load(run_path)
    data = img.get_fdata()  # Extract the data as a numpy array
    
    # Standardize data (z-scoring)
    mean = np.mean(data, axis=3, keepdims=True)
    std = np.std(data, axis=3, keepdims=True)
    standardized_data = (data - mean) / std
    
    # Append standardized data for concatenation
    standardized_fmri_data.append(standardized_data)

# Concatenate the standardized runs along the time dimension
concatenated_data = np.concatenate(standardized_fmri_data, axis=3)

# Create a new NIfTI image for the concatenated data
concatenated_img = nib.Nifti1Image(concatenated_data, img.affine, img.header)

""" Alternative concatenation from Serie 5:
from nilearn.image import concat_imgs, mean_img
fmri_img = concat_imgs(subject_data.func)
"""

# Save the concatenated image
nib.save(concatenated_img, "/path_to_save/concatenated_standardized_fmri.nii.gz")

print("Concatenation complete and saved as concatenated_standardized_fmri.nii.gz")


## 1.3 Pre-processing Data

In [ ]:
# Should it be before concatenation ?

### 1.3.PP1 - Motion Correction

In [ ]:
from fsl.wrappers import mcflirt
"""
dataset_id = 'ds004226'
subject = '001' 

sample_path = "/home/jovyan/Data/dataset"
mkdir_no_exist(sample_path)
bids_root = op.join(os.path.abspath(""),sample_path, dataset_id)
deriv_root = op.join(bids_root, 'derivatives')
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')

subprocess.run(["openneuro-py", "download", "--dataset", dataset_id, "--target-dir", bids_root, "--include", 'sub-{}'.format(subject)], check=True)

#mkdir_no_exist(bids_root)
mkdir_no_exist(deriv_root)
mkdir_no_exist(preproc_root)
mkdir_no_exist(os.path.join(preproc_root, 'sub-001'))
mkdir_no_exist(os.path.join(preproc_root, 'sub-001', 'func'))
"""
path_original_data = os.path.join(bids_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold') # ADapt code
path_moco_data = os.path.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_moco') # ADapt code
mcflirt(infile=path_original_data,o=path_moco_data, plots=True, report=True, dof=6, mats=True)

In [ ]:
print_dir_tree(bids_root, max_depth=5)

In the functional folder, notice that we have two new files:
```
sub-001_task-sitrep_run-01_bold_moco.nii.gz
sub-001_task-sitrep_run-01_bold_moco.par

```

The first one is the corrected EPI time serie, with volumes realigned. The second is a file describing the motion parameters that were used to move each volume. It will be useful very shortly to determine which volume moved by a lot.
Notice as well a new directory!
```
sub-001_task-sitrep_run-01_bold_moco.mat/
```
This directory is full of .MAT files. These are the transformation matrices used for every volume to realign them.

<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b> 💡 Pay attention ! 💡</b></p>
<p style='text-indent: 10px;'>
    The motion parameters and the transformation matrices are related, but they are not exactly the same thing. While you can recover one from the other, it is not trivial. Applying the transformation matrix to a volume will put it 'in alignment' as you've done with FLIRT. However the motion parameters cannot be applied directly. Loosely, the motion parameters describe how you would move if you first applied a rotation along x, then along y, then along y, followed by transition along x, then transition along y, then transition along z. This ordering of transformations is **not** really what happens with the transformation matrices. It is a convention adopted by FSL to make it easier to decouple transformations and rotations in the motion parameter analysis; it is therefore a <b>convenience</b>.
    <u>Do not confuse transformation matrices and motion parameters</u>!</p>
</span>
</div>

In [ ]:
# Looking at the time series
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(path_original_data)
fsleyesDisplay.load(path_moco_data)

### 1.3.PP2 - Smoothing

In [ ]:
cmd = 'fslmaths {} -s {} {}_smoothed-6mm'.format(output_path, 6/2.3548, output_path)
subprocess.run(['fslmaths',output_path, '-s', str(6/2.3548), '{}_smoothed-6mm'.format(output_path)])

In [ ]:
fsleyesDisplay.load(output_path + '_smoothed-6mm')

## Design Matrix

In [ ]:
# Code from Lab05
from nilearn.image import concat_imgs, mean_img
fmri_img = concat_imgs(subject_data.func)
events = pd.read_table(subject_data['events'])
events

In [ ]:
from nilearn.glm.first_level import make_first_level_design_matrix, FirstLevelModel

# Specify what sort of GLM we want (nature of the noise, repetition time of the data and other parameters)
fmri_glm = FirstLevelModel(t_r=7,
                           noise_model='ar1',
                           standardize=False,
                           hrf_model='spm',
                           drift_model=None,
                           high_pass=.01)

# Fit the model to our design and data
fmri_glm = fmri_glm.fit(fmri_img, events)

In [ ]:
from nilearn.plotting import plot_design_matrix
plot_design_matrix(fmri_glm.design_matrices_[0])
plt.show()